In [37]:
# Importing Dependencies
import pandas as pd
import regex as re


In [38]:
# Converting CSV files into Pandas dataframes
df_player = pd.read_csv('Resources/players.csv')
df_player_att = pd.read_csv('Resources/players_attributes.csv')
df_team = pd.read_csv('Resources/team.csv')
df_team_att = pd.read_csv('Resources/team_attributes.csv')
df_fifa_players = pd.read_csv('Resources/fifa_players.csv')

In [39]:
#Getting distinct values based on latest date of recorded statistics of each Team.
df_team_att['date'] = pd.to_datetime(df_team_att['date'])
df_team_att['date'].dt.date
df_team_att.head()

,id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,...,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
0,1,434,9930,2010-02-22,60,Balanced,NaN,Little,50,Mixed,...,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover
1,2,434,9930,2014-09-19,52,Balanced,48.0,Normal,56,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
2,3,434,9930,2015-09-10,47,Balanced,41.0,Normal,54,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
3,4,77,8485,2010-02-22,70,Fast,NaN,Little,70,Long,...,70,Lots,Organised,60,Medium,70,Double,70,Wide,Cover
4,5,77,8485,2011-02-22,47,Balanced,NaN,Little,52,Mixed,...,52,Normal,Organised,47,Medium,47,Press,52,Normal,Cover


In [40]:
# Merging Team information data with Team Attibutes
df_team_combined = pd.merge(df_team, df_team_att, how='inner', on='team_api_id')

# Filtering Data to show only results of the most recent date
latest_indices = df_team_combined.groupby('team_long_name')['date'].idxmax()
latest_team_df = df_team_combined.loc[latest_indices]

# Replacing Null Values using Fillna , Replacing Buildup play dribbling with 0 where class is "Little" indicating that the team does little to no dribling in the build up phase
latest_team_df['buildUpPlayDribbling'].fillna(0, inplace=True)
latest_team_df.head()

C:\Users\rdesh\AppData\Local\Temp\ipykernel_48612\3605989918.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  latest_team_df['buildUpPlayDribbling'].fillna(0, inplace=True)


,id_x,team_api_id,team_fifa_api_id_x,team_long_name,team_short_name,id_y,team_fifa_api_id_y,date,buildUpPlaySpeed,buildUpPlaySpeedClass,...,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
618,16848,8350,29.0,1. FC Kaiserslautern,KAI,636,29,2015-09-10,49,Balanced,...,64,Normal,Organised,41,Medium,51,Press,53,Normal,Cover
539,15624,8722,31.0,1. FC Köln,FCK,425,31,2015-09-10,59,Balanced,...,71,Lots,Organised,42,Medium,56,Press,51,Normal,Cover
606,16239,8165,171.0,1. FC Nürnberg,NUR,920,171,2015-09-10,46,Balanced,...,59,Normal,Organised,30,Deep,40,Press,44,Normal,Cover
612,16243,9905,169.0,1. FSV Mainz 05,MAI,788,169,2015-09-10,60,Balanced,...,44,Normal,Organised,55,Medium,62,Press,47,Normal,Cover
447,11817,8576,614.0,AC Ajaccio,AJA,15,614,2015-09-10,59,Balanced,...,52,Normal,Organised,38,Medium,47,Press,53,Normal,Cover


In [41]:
# Defining function to extract the last name from dataframes to have a common element for merging. 
def extract_last_name(name):
    # Use regex to find the last word in the string, which is assumed to be the last name
    last_name = re.search(r'(\w+)$', name)
    
    # Return the last name if a match is found
    if last_name:
        return last_name.group(1)
    else:
        return None

In [42]:
# This Df is a complete set with progressional growth over the years of each player. 
df_player_comp = pd.merge(df_player,df_player_att, how='left', on='player_api_id' ) 

# Getting distinct values based on latest date of recorded statistics of each player. 
df_player_comp['date'] = pd.to_datetime(df_player_comp['date'])
latest_indices = df_player_comp.groupby('player_name')['date'].idxmax()
latest_value = df_player_comp.loc[latest_indices]

latest_value['birthday'] = pd.to_datetime(latest_value['birthday'])
latest_value['birthday'] = latest_value['birthday'].dt.date

In [43]:
# Using function to extract last name using Regex. This way we have a common element in both datasets and can use that to merge. 
latest_value['last_name'] = latest_value['player_name'].apply(extract_last_name)
latest_value


,id_x,player_api_id,player_name,player_fifa_api_id_x,birthday,height,weight,id_y,player_fifa_api_id_y,date,...,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,last_name
0,1,505942,Aaron Appindangoye,218353,1992-02-29,182.88,187,1,218353,2016-02-18,...,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0,Appindangoye
5,2,155782,Aaron Cresswell,189615,1989-12-15,170.18,146,6,189615,2016-04-21,...,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0,Cresswell
38,3,162549,Aaron Doran,186170,1991-05-13,170.18,163,39,186170,2016-01-07,...,61.0,23.0,22.0,24.0,16.0,11.0,12.0,9.0,13.0,Doran
64,4,30572,Aaron Galindo,140161,1982-05-08,182.88,198,65,140161,2016-04-21,...,37.0,72.0,71.0,68.0,15.0,12.0,13.0,12.0,11.0,Galindo
87,5,23780,Aaron Hughes,17725,1979-11-08,182.88,154,88,17725,2015-12-24,...,45.0,75.0,73.0,71.0,8.0,6.0,16.0,12.0,11.0,Hughes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183953,11073,36491,Zsolt Low,111191,1979-04-29,180.34,154,183954,111191,2011-02-22,...,61.0,67.0,71.0,59.0,15.0,13.0,5.0,6.0,6.0,Low
183960,11074,35506,Zurab Khizanishvili,47058,1981-10-06,185.42,172,183961,47058,2011-02-22,...,45.0,65.0,66.0,65.0,14.0,7.0,15.0,10.0,12.0,Khizanishvili
183968,11075,39902,Zvjezdan Misimovic,102359,1982-06-05,180.34,176,183969,102359,2012-08-31,...,78.0,22.0,31.0,30.0,9.0,13.0,8.0,15.0,5.0,Misimovic
41617,2483,11833,de Oliveira Cleber Monteiro,163001,1980-05-23,182.88,170,41618,163001,2012-02-22,...,51.0,53.0,64.0,61.0,15.0,15.0,15.0,12.0,14.0,Monteiro


In [44]:
# Using function to extract last name using Regex. This way we have a common element in both datasets and can use that to merge. 
df_fifa_players['last_name'] = df_fifa_players['name'].apply(extract_last_name)

In [45]:
# Renaming Column for consistency
df_fifa_players.rename(columns= {'birth_date' : 'birthday'}, inplace=True)

In [46]:
# combining dataframes to cummalate all relevant columns 
df_combined = pd.merge(latest_value, df_fifa_players, how= 'inner', on='last_name')
df_combined.head()

,id_x,player_api_id,player_name,player_fifa_api_id_x,birthday_x,height,weight,id_y,player_fifa_api_id_y,date,...,long_shots_y,aggression_y,interceptions_y,positioning_y,vision_y,penalties_y,composure,marking_y,standing_tackle_y,sliding_tackle_y
0,2,155782,Aaron Cresswell,189615,1989-12-15,170.18,146,6,189615,2016-04-21,...,62,73,75,60,68,59,71,72,78,79
1,5,23780,Aaron Hughes,17725,1979-11-08,182.88,154,88,17725,2015-12-24,...,30,70,65,41,35,38,49,68,70,64
2,5,23780,Aaron Hughes,17725,1979-11-08,182.88,154,88,17725,2015-12-24,...,27,69,62,45,38,45,56,64,63,61
3,5,23780,Aaron Hughes,17725,1979-11-08,182.88,154,88,17725,2015-12-24,...,26,66,63,29,41,45,70,65,65,63
4,5,23780,Aaron Hughes,17725,1979-11-08,182.88,154,88,17725,2015-12-24,...,37,63,64,23,46,41,63,66,65,62


In [47]:
# Dropped duplicate values from dataframe
df_combined.drop_duplicates(subset='player_name', inplace=True)


In [48]:
# Dropping Duplicate and irrelevant columns
df_combined.drop(columns=['player_fifa_api_id_x','weight', 'id_y', 'player_fifa_api_id_y','date','attacking_work_rate', 'defensive_work_rate','last_name', 'name',
       'full_name', 'birthday_y','height_cm','overall_rating_y', 'potential_y','preferred_foot_y',
       'international_reputation(1-5)', 'weak_foot(1-5)', 'skill_moves(1-5)',
       'body_type', 'release_clause_euro', 'national_team', 'national_rating',
       'national_team_position', 'national_jersey_number', 'crossing_y',
       'finishing_y', 'heading_accuracy_y', 'short_passing_y', 'volleys_y',
       'dribbling_y', 'curve_y', 'freekick_accuracy', 'long_passing_y',
       'ball_control_y', 'acceleration_y', 'sprint_speed_y', 'agility_y',
       'reactions_y', 'balance_y', 'shot_power_y', 'jumping_y', 'stamina_y',
       'strength_y', 'long_shots_y', 'aggression_y', 'interceptions_y',
       'positioning_y', 'vision_y', 'penalties_y', 'composure', 'marking_y',
       'standing_tackle_y', 'sliding_tackle_y'], inplace=True)

In [49]:
# Removed Suffixes ('_x' and '_y') created by the merges 
df_combined.rename(columns=lambda x: x.replace('_x', '').replace('_y', ''), inplace=True)
df_combined.columns

Index(['id', 'player_api_id', 'player_name', 'birthday', 'height',
       'overall_rating', 'potential', 'preferred_foot', 'crossing',
       'finishing', 'heading_accuracy', 'short_passing', 'volleys',
       'dribbling', 'curve', 'free_kick_accuracy', 'long_passing',
       'ball_control', 'acceleration', 'sprint_speed', 'agility', 'reactions',
       'balance', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots',
       'aggression', 'interceptions', 'positioning', 'vision', 'penalties',
       'marking', 'standing_tackle', 'sliding_tackle', 'gk_diving',
       'gk_handling', 'gk_kicking', 'gk_positioning', 'gk_reflexes', 'age',
       'weight_kgs', 'positions', 'nationality', 'value_euro', 'wage_euro'],
      dtype='object')

In [50]:
# Removed Null values within the dataframe.
df_cleaned = df_combined.dropna(subset=['wage_euro','value_euro','volleys','curve','agility','balance','jumping'])


In [51]:
# final check for null values
df_cleaned.isnull().sum()

id                    0
player_api_id         0
player_name           0
birthday              0
height                0
overall_rating        0
potential             0
preferred_foot        0
crossing              0
finishing             0
heading_accuracy      0
short_passing         0
volleys               0
dribbling             0
curve                 0
free_kick_accuracy    0
long_passing          0
ball_control          0
acceleration          0
sprint_speed          0
agility               0
reactions             0
balance               0
shot_power            0
jumping               0
stamina               0
strength              0
long_shots            0
aggression            0
interceptions         0
positioning           0
vision                0
penalties             0
marking               0
standing_tackle       0
sliding_tackle        0
gk_diving             0
gk_handling           0
gk_kicking            0
gk_positioning        0
gk_reflexes           0
age             

In [52]:
# Converting Birthday Colum to datetime format. 
df_cleaned['birthday'] = df_cleaned['birthday'].astype('datetime64[ns]')

C:\Users\rdesh\AppData\Local\Temp\ipykernel_48612\2959105930.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['birthday'] = df_cleaned['birthday'].astype('datetime64[ns]')


In [63]:
# Filtering Dataframe to contain information of Defensive players
df_defensive = df_cleaned[df_cleaned['positions'].str.contains('LB|RB|CB|LWB|RWB')]

# Filtering Dataframe to keep columns relevant to Defenders
df_defensive = df_defensive[['id','player_name','age','overall_rating','potential','aggression', 'interceptions', 
                             'positioning','marking','long_shots', 'standing_tackle','short_passing','long_passing', 'sliding_tackle','value_euro', 'wage_euro']]
df_defensive.head()

,id,player_name,age,overall_rating,potential,aggression,interceptions,positioning,marking,long_shots,standing_tackle,short_passing,long_passing,sliding_tackle,value_euro,wage_euro
0,2,Aaron Cresswell,29,74.0,76.0,68.0,67.0,60.0,76.0,62.0,75.0,71.0,68.0,78.0,6000000.0,63000.0
1,5,Aaron Hughes,26,70.0,70.0,66.0,68.0,29.0,75.0,26.0,73.0,65.0,58.0,71.0,875000.0,6000.0
6,6,Aaron Hunt,28,77.0,77.0,59.0,55.0,77.0,28.0,76.0,34.0,79.0,70.0,35.0,875000.0,18000.0
15,10,Aaron Meijers,31,69.0,69.0,73.0,67.0,65.0,67.0,61.0,68.0,70.0,53.0,65.0,750000.0,5000.0
33,23,Abdelhamid El Kaoutari,28,73.0,75.0,85.0,72.0,48.0,75.0,38.0,76.0,64.0,56.0,77.0,950000.0,4000.0


In [54]:
# Filter Dataframe to contain information of Attacking Players
df_attackers = df_cleaned[df_cleaned['positions'].str.contains('CM|CAM|RM|LM|ST|CF|\bLW\b|\bRW\b')]

# Filtering Dataframe to keep columns relevant to Attackers
df_attackers = df_attackers[['id','player_name','age','nationality', 'value_euro', 'wage_euro','overall_rating', 'potential','crossing',
       'finishing', 'heading_accuracy', 'short_passing', 'volleys',
       'dribbling', 'curve', 'free_kick_accuracy', 'long_passing',
       'ball_control','shot_power', 'long_shots']]
df_attackers.head()

,id,player_name,age,nationality,value_euro,wage_euro,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,shot_power,long_shots
11,8,Aaron Lennon,31,England,7000000.0,48000.0,77.0,77.0,78.0,66.0,28.0,76.0,68.0,84.0,63.0,55.0,54.0,81.0,62.0,57.0
17,12,Aaron Mooy,28,Australia,10500000.0,50000.0,75.0,78.0,74.0,69.0,46.0,79.0,65.0,73.0,73.0,78.0,79.0,73.0,70.0,72.0
18,15,Aaron Ramsey,28,Wales,23000000.0,130000.0,82.0,85.0,75.0,75.0,58.0,83.0,79.0,81.0,70.0,70.0,80.0,82.0,81.0,75.0
19,17,Aaron Taylor-Sinclair,29,England,6500000.0,48000.0,64.0,68.0,62.0,26.0,59.0,62.0,25.0,66.0,57.0,58.0,47.0,62.0,59.0,30.0
24,18,Aaron Wilbraham,39,England,110000.0,1000.0,65.0,65.0,49.0,67.0,78.0,68.0,56.0,49.0,43.0,42.0,39.0,65.0,69.0,47.0


In [55]:
#Filtering Data to contain information of physical attributes
df_physical = df_cleaned[['id','player_name','age','nationality', 'value_euro', 'wage_euro','height','weight_kgs','reactions',
       'balance','jumping', 'stamina', 'strength','aggression']]

df_physical.head()

,id,player_name,age,nationality,value_euro,wage_euro,height,weight_kgs,reactions,balance,jumping,stamina,strength,aggression
0,2,Aaron Cresswell,29,England,6000000.0,63000.0,170.18,66.2,67.0,90.0,85.0,79.0,56.0,68.0
1,5,Aaron Hughes,26,Wales,875000.0,6000.0,182.88,81.2,68.0,52.0,67.0,60.0,75.0,66.0
6,6,Aaron Hunt,28,England,875000.0,18000.0,182.88,71.2,80.0,72.0,65.0,75.0,65.0,59.0
11,8,Aaron Lennon,31,England,7000000.0,48000.0,165.10,63.0,74.0,92.0,71.0,75.0,56.0,57.0
15,10,Aaron Meijers,31,Netherlands,750000.0,5000.0,175.26,77.1,70.0,72.0,70.0,78.0,66.0,73.0


In [60]:
# Filter the DataFrame to contain data for goal keepers only
df_gk = df_cleaned[df_cleaned['positions'].str.contains('GK')]

# Filtering Dataframe to keep columns relevant to Goalkeepers
df_gk = df_gk[['id', 'player_name','age', 'nationality','height', 'weight_kgs','positions', 'value_euro', 'wage_euro','overall_rating','potential', 'agility','positioning', 'reactions', 'jumping', 'strength',
       'balance','stamina']]
df_gk.sample(10)

,id,player_name,age,nationality,height,weight_kgs,positions,value_euro,wage_euro,overall_rating,potential,agility,positioning,reactions,jumping,strength,balance,stamina
18556,10862,Xavier Chen,22,China PR,175.26,83.0,GK,130000.0,1000.0,67.0,67.0,74.0,48.0,63.0,68.0,59.0,76.0,65.0
239,118,Adam Matthews,25,England,177.80,78.0,GK,725000.0,3000.0,73.0,77.0,76.0,63.0,73.0,71.0,70.0,73.0,75.0
9887,5578,Justo Villar,21,Spain,180.34,78.9,GK,950000.0,4000.0,75.0,75.0,52.0,14.0,69.0,56.0,62.0,68.0,22.0
2018,1070,Azbe Jug,27,Slovenia,190.50,88.9,GK,575000.0,2000.0,66.0,72.0,42.0,11.0,65.0,57.0,74.0,40.0,36.0
4491,2362,David Gonzalez,20,Chile,193.04,83.0,GK,150000.0,1000.0,69.0,69.0,44.0,15.0,63.0,62.0,72.0,26.0,26.0
14063,8314,Parfait Mandanda,20,France,182.88,77.1,GK,800000.0,2000.0,68.0,70.0,56.0,11.0,65.0,75.0,73.0,42.0,18.0
2064,1099,Baptiste Reynet,28,France,185.42,84.8,GK,8500000.0,24000.0,71.0,74.0,43.0,12.0,75.0,62.0,68.0,24.0,34.0
851,456,Alexander Gonzalez,20,Chile,177.80,83.0,GK,150000.0,1000.0,66.0,74.0,76.0,61.0,62.0,90.0,74.0,74.0,80.0
18592,10873,Ximo Navarro,33,Argentina,177.80,87.1,GK,2000000.0,12000.0,74.0,76.0,73.0,61.0,72.0,79.0,71.0,73.0,84.0
1828,953,Arkadiusz Malarz,38,Poland,187.96,86.2,GK,120000.0,3000.0,65.0,65.0,21.0,13.0,57.0,35.0,44.0,43.0,28.0


In [59]:
#exporting Dataframes to Csv
df_player.to_csv('Resources/players.csv', index=False)
df_player_att.to_csv('Resources/players_attributes.csv', index=False)
df_team.to_csv('Resources/team.csv', index=False)
df_team_att.to_csv('Resources/team_attributes.csv', index=False)
df_cleaned.to_csv('Resources/cleaned_player_data.csv', index=False)
latest_team_df.to_csv('Resources/cleaned_team_data.csv', index=False)
df_gk.to_csv('Resources/goalkeeper_data.csv', index=False)
df_attackers.to_csv('Resources/attackers_data.csv', index=False)
df_defensive.to_csv('Resources/defensive_data.csv', index=False)
df_physical.to_csv('Resources/physical_data.csv', index=False)
